In [1]:
# import required libraries for EvoMan
from evoman.environment import Environment
from demo_controller import player_controller

# imports other libs
import numpy as np
import os
import random
import sys
import matplotlib.pyplot as plt

<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


pygame 2.6.0 (SDL 2.30.7, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
# set up simulation
def simulation(env, genome):
    f,p,e,t = env.play(pcont=genome)
    return f

In [2]:
x = np.random.uniform(0, 1, size=(5, 10))
print(x)

[[0.5135738  0.01740534 0.31979974 0.63485364 0.87506323 0.53487383
  0.35766153 0.48304208 0.17433856 0.26757185]
 [0.59092372 0.01626029 0.88489751 0.46508939 0.60397244 0.52323487
  0.11535087 0.82564895 0.76109679 0.60594903]
 [0.88779897 0.4564099  0.82003056 0.08596859 0.2460258  0.55125398
  0.366769   0.89549842 0.1233953  0.47657068]
 [0.10051341 0.91352921 0.77680514 0.61202317 0.7573202  0.9259962
  0.32059311 0.48113806 0.58971181 0.37480141]
 [0.92084632 0.94858903 0.22929616 0.49382333 0.32285003 0.11176785
  0.9916192  0.44791698 0.50346184 0.86104026]]


## Support functions 

In [ ]:
def get_fitness():
    pass

def pop_init(n):
    """K-Tournament initialization"""
    population = []
    genome_size = (20 + 1) * 10 + (10 + 1) * 5
    # generate population from iterativer tournaments  
    for _ in range(n):
        # create initalization tournamentn
        tournament = np.random.uniform(-1, 1, size=(5, genome_size))
        tournament_fitness = get_fitness(tournament)
        # extract tournament winner and add to population
        best = tournament[np.argmax(tournament_fitness)]
        population.append(best)
    
    return np.array(population)



def local_search():
    pass

def survival_selection():
    pass

## Evo Strategies

In [ ]:
def evo_strategies(pop_size, m_rate, p_crossover, mutation_operator, max_gen, **kwargs):
    """Evolution strategy algorithm made for testing different mutation operators"""

    # initialize population
    population = pop_init(pop_size)
    fit_values = get_fitness()

    # get best individuals
    idx = np.argmax(fit_values)
    best_ind = [population[idx]]
    best_fit = [fit_values[idx]]

    # start evos trategies
    for _ in range(max_gen):

        # mutate population and evaluate
        mutated = mutation_operator(population, m_rate, **kwargs)
        fit_mutated = get_fitness(mutated)
        # memetic algorithm -> life improvement of individuals 
        improved_mutated, fit_imporved = local_search(mutated, fit_mutated)
        # survival selection 
        population, fit_values = survival_selection(population, fit_values, improved_mutated, fit_imporved)
        
        # record best solution 
        idx = np.argmax(fit_values)
        gen_best = population[idx]
        gen_best_fitness = fit_values[idx]
        # check if gen best is highest 
        if gen_best_fitness > best_fit[-1]:
            best_ind.append(gen_best)
            best_fit.append(gen_best_fitness)
        else:
            best_ind.append(best_ind[-1])
            best_fit.append(best_fit[-1])

        # termination condition
        if best_fit[-1] > 95:
            break
    
    return best_ind, best_fit 